# Learning a Gaussian with tensorflow


In [1]:
import numpy as np
import pandas as pd
import scipy.io
import scipy.sparse
import scipy.stats
import tensorflow as tf
import tensorflow_probability as tfp
from scipy.io import loadmat

tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers
import matplotlib.pyplot as plt
from PIL import Image


2022-10-25 15:23:35.261860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-25 15:23:35.491593: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/include:/usr/local/cuda-11.0/lib64:
2022-10-25 15:23:35.491630: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-25 15:23:35.531187: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regi

We load the iris data set.


In [7]:
from sklearn.datasets import load_iris

data = load_iris()["data"]

X = ((data - np.mean(data, 0)) / np.std(data, 0)).astype(np.float32)
n = X.shape[0]  # number of observations
p = X.shape[1]  # number of features


Learn a Gaussian model $p(x) = \mathcal{N}(x|\mu,\Sigma)$ using Tensorflow!


In [97]:
def log_lik(X):
  gaussian_diag_cov = tfd.MultivariateNormalDiag(loc=mu)  # cov = identity
  outputs = gaussian_diag_cov.log_prob(X)
  return tf.reduce_mean(outputs)


def train_step(X: np.ndarray, params: list):
  """Data is a batch of data points"""
  with tf.GradientTape() as tape:
    loss = -log_lik(X)

  gradients = tape.gradient(loss, params)

  optim = tfk.optimizers.Adam(learning_rate=0.005)
  optim.apply_gradients(zip(gradients, params))


mu = tf.Variable(initial_value=tf.random.normal([p]), trainable=True)
train_dataset = (
  tf.data.Dataset.from_tensor_slices(tensors=X).shuffle(n).batch(batch_size=16)
)

nb_epochs = 200
for epoch in range(nb_epochs):
  for batch in train_dataset:
    train_step(X=batch, params=[mu])
  print(log_lik(X=batch), "\t", mu.numpy())

mu


tf.Tensor(-11.574745, shape=(), dtype=float32) 	 [-1.2113843  0.2434182  2.181826   1.8863592]
tf.Tensor(-11.198485, shape=(), dtype=float32) 	 [-1.1613843   0.21341956  2.1318274   1.8363593 ]
tf.Tensor(-8.63192, shape=(), dtype=float32) 	 [-1.1113844   0.18341915  2.0818286   1.7863593 ]
tf.Tensor(-12.664432, shape=(), dtype=float32) 	 [-1.0613844   0.15342006  2.0318298   1.7363594 ]
tf.Tensor(-11.085523, shape=(), dtype=float32) 	 [-1.0113845   0.11342181  1.9818306   1.6863594 ]
tf.Tensor(-9.711522, shape=(), dtype=float32) 	 [-0.96138453  0.09342293  1.9318306   1.6363595 ]
tf.Tensor(-7.8157907, shape=(), dtype=float32) 	 [-0.9113846   0.08342185  1.8818307   1.5863595 ]
tf.Tensor(-10.728185, shape=(), dtype=float32) 	 [-0.8613849   0.05342279  1.8318307   1.5363595 ]
tf.Tensor(-7.565163, shape=(), dtype=float32) 	 [-0.81138504  0.04341358  1.7818308   1.4863596 ]
tf.Tensor(-5.8078246, shape=(), dtype=float32) 	 [-0.76138514  0.03341419  1.7318308   1.4363596 ]
tf.Tensor(-7.40663

<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([-0.01121331, -0.01656599, -0.01778212,  0.00649727], dtype=float32)>